## Importing the libraries

In [1]:
# imports

import os
import json  # 2.0.9
from openai import OpenAI # 1.61.1
import gradio as gr # 5.7.1
import time
from generate_flux import generate_flux_schnell
from generate_image import generate_image
from extract_sections import extract_sections

## Configuring the system instructions

In [2]:
system_message = "You are a Comic artist called ComicCrafterAI. "
system_message += "Your Goal is to generate a comic book style ultra short story based on user-provided prompts in not more than 50 words "
system_message += "The Story is divided into four parts: Introduction: , Storyline: , Climax: ,  Moral: "
system_message += "Generate the paragraph of each section and do not generate the content of each section in the form of panels"

## Defining the API using OPENAI API interface

In [3]:
gemma2b = OpenAI(
    base_url="http://127.0.0.1:5000/v1",
    api_key="lm-studio"
)

## Our local chat streaming function
### It utilises 'gemma-2-2b-it' instruction model which is best suited for the system or user defined chatbot interaction. It streams the response output and extracts the structured sections such as Introduction, Storyline, Climax and Moral through ' extract_sections.py ' module. The ' generate_image.py ' module takes each of the extracted content and provides the desired image output.

In [ ]:
def chat_local_streaming(message, image_function):
    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": message}]
    response = gemma2b.chat.completions.create(
        model="gemma-2-2b-it",
        messages=messages,
        temperature=0.9,
        stream=True  
    )
    reply = """ """
    yield reply, gr.update(value=None, visible=True, label="Processing...this may take upto 5 mins")  
    
    for chunk in response:
        text = chunk.choices[0].delta.content
        if text:
            reply += text
            yield reply, gr.update(value=None, visible=True, label="Processing...this may take upto 5 mins")  

    sections = extract_sections(reply)
    introduction = sections.get("introduction", "")
    storyline = sections.get("storyline", "")
    climax = sections.get("climax", "")
    moral = sections.get("moral", "")
    
    image_generator = generate_flux_schnell if image_function == "generate_flux_schnell" else generate_image
    
    images = [
        image_generator(introduction), 
        image_generator(storyline),
        image_generator(climax),
        image_generator(moral)
    ]
    
    yield reply, gr.update(value=images, visible=True, label="Generated Comic Images")  


## We combine our functions and defined modules in a nice and user friendly Gradio interface

In [7]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_outputs = gr.Gallery(height=500, label="Generated Comic Images", visible=True, columns=2) 
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
        image_function_selector = gr.Dropdown(["generate_flux_schnell", "generate_image"], label="Select Image Generation Function")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history, image_function):
        history += [{"role": "user", "content": message}]
        response_gen = chat_local_streaming(message, image_function)
        reply = ""
        images = []
        for output in response_gen:
            if isinstance(output, tuple):
                reply, images = output
            else:
                reply = output
            yield "", history + [{"role": "assistant", "content": reply}], images

    entry.submit(
        do_entry,
        inputs=[entry, chatbot, image_function_selector],
        outputs=[entry, chatbot, image_outputs]
    )
    clear.click(lambda: ([], gr.update(value=None, visible=False)), inputs=None, outputs=[chatbot, image_outputs], queue=False)

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
